# Second-Order Methods in TensorFlow - Part 2

Now we'd like to actually implement a second-order method. Here, we implement a very silly version of Newton's method.

In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

from collections import namedtuple

## Minimizing Quadratic Forms with Gradient Descent and Newton's Method

In [2]:
QuadraticFormMinimizer = namedtuple("QuadraticFormMinimizer", ["graph", "graph_dictionary"])

In [3]:
def make_quadratic_form_minimizer(matrix, initial_values, hyperparameters):
    assert matrix.shape[0] == matrix.shape[1], "only square matrices can be quadratic forms"
    assert matrix.shape[0] == len(initial_values), "initial_values and matrix must match shape"
    
    dimension = matrix.shape[0]
    
    graph = tf.Graph()
    
    with graph.as_default():
        quadratic_form = tf.constant(matrix, name='quadratic_form')
        
        inputs = tf.get_variable("inputs", shape=[dimension], dtype=tf.float64,
                                initializer = tf.constant_initializer(initial_values))

        input_vector = tf.reshape(inputs, (dimension, 1))
    
        output = tf.squeeze(tf.matmul(input_vector,
                                  tf.matmul(quadratic_form, input_vector),
                                      transpose_a=True,
                                  name='output'),
                        name='squeezed_output')
        
        gradients = tf.gradients(output, inputs, name="gradients")
        
        hessian_matrix = tf.hessians(output, inputs, name="hessian_matrix")[0]
        
        fudging_vector = tf.constant([hyperparameters["fudge_factor"]]*N, dtype=tf.float64,
                                     name="fudging_vector")
        fudged_hessian = tf.add(tf.diag(fudging_vector),
                                        hessian_matrix, name ="fudged_hessian")
        inverse_hessian = tf.matrix_inverse(fudged_hessian, name="inverse_fudged_hessian")

        gradient_descent = tf.train.GradientDescentOptimizer(hyperparameters["learning_rate"])
        step_gradient_descent = gradient_descent.minimize(output)
        
        newton_base = tf.train.GradientDescentOptimizer(hyperparameters["newton_rate"])
        gd_grads_and_vars = newton_base.compute_gradients(output, inputs)
        step_newton = add_step_newton(newton_base, gd_grads_and_vars, inverse_hessian)
        
        graph_dictionary = {"inputs": inputs,
                           "output": output,
                           "gradients": gradients,
                           "hessian": hessian_matrix,
                            "step_gradient_descent": step_gradient_descent,
                            "step_newton": step_newton
                           }
    
    return QuadraticFormMinimizer(graph, graph_dictionary)

def add_step_newton(gradient_descent, gd_grads_and_vars, inverse_hessian):
    gd_gradients, gd_variables = gd_grads_and_vars[0]
    gd_gradient_vector = tf.expand_dims(gd_gradients, name="gradient_vector", axis=1)

    newton_gradient_vector = tf.matmul(inverse_hessian, gd_gradient_vector,
                                           name="newton_gradient_vector")
    newton_gradients = tf.squeeze(newton_gradient_vector)
      
    newton_grads_and_vars = [(newton_gradients, gd_variables)]

    step_newton = gradient_descent.apply_gradients(newton_grads_and_vars)
    
    return step_newton

In [4]:
def run_minimizer(quadratic_form_minimizer, algorithm, num_steps):
    graph, graph_dictionary = quadratic_form_minimizer
    
    with graph.as_default():
        with tf.Session() as sess:
            graph_dictionary["inputs"].initializer.run()
            for _ in range(num_steps):
                sess.run(graph_dictionary["step_"+algorithm])
            output = sess.run(graph_dictionary["output"])
            values = graph_dictionary["inputs"].eval()
    
    return output, values

## Testing with Identity Matrix

The identity matrix also makes a good test for the Newton method, since, if we set the `newton_rate`, or the learning rate of the Newton method, to exactly $1$, (and if we turn off Hessian-fudging, to be explained in a second) then we should get convergence in a single step.

The value that minimizes the identity matrix is the same that minimizes any positive definite quadratic form: $\mathbf{0}$, the zero vector.

In [6]:
N = 2

identity_matrix = np.eye(N)
#initial_values = np.atleast_2d(np.sqrt([1/2,1/2])).T
initial_values = np.random.standard_normal(size=N)

identity_quadratic_form_minimizer = make_quadratic_form_minimizer(identity_matrix, initial_values,
                                                                 {"learning_rate":0.1,
                                                                 "newton_rate":1,
                                                                 "fudge_factor":0.0})

In [8]:
final_output, final_parameters = run_minimizer(identity_quadratic_form_minimizer, "newton", 1)

assert final_output == 0
assert np.array_equal(final_parameters, [0,0])

In contrast, gradient descent takes more than one step to even approximately minimize this quadratic form.

In [9]:
final_output, final_parameters = run_minimizer(identity_quadratic_form_minimizer, "gradient_descent", 1)

final_output, final_parameters

(0.60451602848147779, array([-0.31407713,  0.7112465 ]))

## Random Positive Definite Matrix

We expect that the Hessians of neural newtorks will behave approximately as if they were drawn from the Wishart distribution, which is generated by taking the outer product of a Gaussian random vector with itself.

The spectra of these matrices should roughly follow the Marçenko-Pastur distribution, which frequently produces eigenvalues that are close to or exactly $0$.

This is a problem for the Newton method, which relies on inverting the Hessian -- a Hessian with some eigenvalues equal to $0$ is not invertible!

We get around this with a *fudge factor*. Just adding a very small number to the diagonal of the Hessian adds that small number to all of its eigenvalues. The Newton updates are no longer exact, but at least they're not not-a-number.

Again, we generically see better performance for one step of the Newton method than of gradient descent.

In [10]:
N = 5

self_outer_product = lambda x: x@x.T
wishart_random_matrix = self_outer_product(np.random.standard_normal(size=(N,1)))

initial_values = np.random.standard_normal(size=N)

wishart_quadratic_form_minimizer = make_quadratic_form_minimizer(wishart_random_matrix, initial_values,
                                                                 {"learning_rate":0.1,
                                                                 "newton_rate":1,
                                                                 "fudge_factor":10e-8})

In [11]:
run_minimizer(wishart_quadratic_form_minimizer, "newton", 1)

(7.3880575715118251e-16,
 array([-1.23586491, -2.76591031,  0.40219102,  1.18296039, -0.53176368]))

In [12]:
run_minimizer(wishart_quadratic_form_minimizer, "gradient_descent", 1)

(0.00057802399664614798,
 array([-1.24228301, -2.76263253,  0.40258752,  1.18660558, -0.52548753]))

## Random Negative Definite Matrix

The Newton method is designed for convex problems. For non-convex problems, it does exactly the opposite of what it should: when the gradient says go down, it actually says go up! This causes it to be attaracted to maxima in this case. For a negative definite quadratic form, this maximum is at $0$.

So if we try to run the Newton method in such a case, we should actually "pessimize", rather than optimize, the function. Gradient descent, even with just one step, will almost surely do better.

For our work, this is a *good thing*, since it means we've found a place where the gradient is close to a $0$: a critical point that is not a minimum (or even a saddle!).

In [13]:
N = 5

self_outer_product = lambda x: x@x.T
negative_wishart_random_matrix = -1*self_outer_product(np.random.standard_normal(size=(N,1)))

initial_values = np.random.standard_normal(size=N)

negative_wishart_quadratic_form_minimizer = make_quadratic_form_minimizer(
                                                    negative_wishart_random_matrix, initial_values,
                                                                 {"learning_rate":0.1,
                                                                 "newton_rate":1,
                                                                 "fudge_factor":10e-8})

In [14]:
run_minimizer(negative_wishart_quadratic_form_minimizer, "newton", 1)

(-3.6472003033283916e-15,
 array([ 1.36625033, -1.22092162,  0.70182029,  1.471215  , -0.66823492]))

In [15]:
run_minimizer(negative_wishart_quadratic_form_minimizer, "gradient_descent", 1)

(-35.439952592293167,
 array([ 1.72377876,  0.62870828,  1.97252841,  0.97166004, -3.08194262]))